# Распознавание цифр

В этот раз мы будем использовать датасет MNIST, который содержить около 60 000 картинок с цифрами, которые были написаны отруки. Каждая картинка в нём имеет размер 28х28 пикселей. Цифры: от 0 до 9. 

То есть в этом случае мы имеет дело с мультиклассовой классификацией. 

> MNIST расшифровывается как Modified National Institute of Standart and Technology. 

Сначала импортируем все необходимые библиотеки. 

> Для распознавания будем использовать torch 

In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.transforms import transforms
import torch.nn.functional as F

Инициализируем параметры 

In [ ]:
CONFIG = { 
    "input_size": 784, # 28x28 
    "hidden_size_1": 200, # размер 1-го скрытого слоя
    "hidden_size_2": 150, # размер 2-го скрытого слоя
    "hidden_size_3": 100, # размер 3-го скрытого слоя
    "hidden_size_4": 80,  # размер 4-го скрытого слоя 
    "output": 10,         # кол-во выходов сети (т.к. цифры от 0 до 9)
    "bach_size": 100,
    "lr_rate": 0.01
}

Загружаем данные, повезло, что в библиотеке torchvision уже есть функция, которая всё сделает за нас. 

In [ ]:
train_data = torchvision.datasets.MNIST(
    "mnist_content", train=True, transform=transforms.ToTensor(), download=True
)

In [ ]:
test_data = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=transforms.ToTensor(), download=False
)

Создаём лоядеры данных

In [ ]:
train_dataloader=torch.utils.data.DataLoader(dataset=train_data, 
                                  batch_size=CONFIG["bach_size"],shuffle=True)
train_dataloader=torch.utils.data.DataLoader(dataset=test_data, 
                                  batch_size=CONFIG["bach_size"],shuffle=True)

In [ ]:
data=iter(train_dataloader)
samples,labels=next(data)
print(f"number of samples{samples.shape}")
print(f"number of labels {labels.shape}")

Давайте посмотрим на картинки 

In [ ]:
plt.figure(figsize=(10,8))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(samples[i][0],cmap='BuPu')
plt.show()

## Полезные ссылки 

* [MNIST Handwritten Digit Recognition Using Pytorch](https://medium.com/analytics-vidhya/training-mnist-handwritten-digit-data-using-pytorch-5513bf4614fb)